In [109]:
import numpy as np
import pandas as pd
import nltk
import string
import re
import matplotlib.pyplot as plt
from scipy import stats
from keras.preprocessing import sequence, text
from keras.layers import  Input, Dense, Flatten, Add, LSTM, GlobalAveragePooling1D,SpatialDropout1D, Bidirectional,\
    BatchNormalization, Concatenate, Dropout, Activation, Input, Embedding, Conv1D, MaxPooling1D, \
    GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.models import Model, load_model
import tensorflow as tf
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard, Callback
import keras.backend as K
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [129]:
from sklearn.metrics import roc_auc_score
class ROCCallBack(Callback):
    def __init__(self,validation_data):
        super().__init__()
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_epoch_end(self, epoch, logs={}):
        y_pred_val = self.model.predict(self.x_val)
        print('\nroc-auc_val: %s' % (str(round(roc_auc_score(self.y_val, y_pred_val),4))))

In [4]:
def flatten(x):
    if isinstance(x, (np.ndarray, list, tuple, pd.Series)):
        lst = []
        for i in x:
            lst += flatten(i)
        return lst
    else:
        return [x]

#### Load Data

In [5]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [6]:
output_names = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [83]:
tok=text.Tokenizer(filters = '!"#$%&()*+,-./:;<=>?@[\\]^_\'`{|}~\t\n', lower=True)
tok.fit_on_texts(np.concatenate((train.comment_text.values, test.comment_text.values)))

#### Load GloVe

In [84]:
f = open('data/glove.42B.300d.txt', 'r', encoding = 'utf-8')

In [85]:
all_unique_tokens = tok.word_index.keys()

In [86]:
embeddings = {}
for line in f:
    values = line.split()
    word = values[0]
    # Whole GloVe embeddings doesn't fit in my GPU memory, so only take words which appear in data for now. 
    # Can always swap weights for embedding layer after model training
    if word in all_unique_tokens:
        coefs = np.array(values[1:], dtype = 'float32')
        embeddings[word] = coefs

In [87]:
for i in list(tok.word_index.keys()):
    if i not in embeddings.keys():
        del tok.word_index[i]
for counter, i in enumerate(tok.word_index.keys()):
    tok.word_index[i] = counter+1

In [88]:
idx2word = {b:a for a,b in tok.word_index.items()}
idx2word[0] = '<UNK>'
word2idx = defaultdict(lambda x: '<UNK>', tok.word_index)
embeddings['<UNK>'] = np.zeros((300,))

#### Data Processing

In [89]:
train['toks'] = tok.texts_to_sequences(train.comment_text.values)
test['toks'] = tok.texts_to_sequences(test.comment_text.values)

In [90]:
vocab_size = len(embeddings)
max_len = 300
n_factors = 300

In [91]:
def create_emb():
    emb = np.zeros((vocab_size+1,n_factors), dtype = 'float32')
    for i in range(0, vocab_size):
        word = idx2word[i]
        emb[i,:] = embeddings[word] #each row is a word
    return emb

In [92]:
emb = create_emb()

In [93]:
emb.shape

(190324, 300)

In [97]:
# train val  split
np.random.seed(10)
indexTrain = np.random.choice(range(train.shape[0]), size = int(0.9*train.shape[0]), replace = False)
indexVal = list(set(range(train.shape[0])) - set(indexTrain))
traindf = train.loc[indexTrain]
valdf = train.loc[indexVal]

In [98]:
dataInputTrain=sequence.pad_sequences(traindf.toks,maxlen=max_len)
dataInputVal=sequence.pad_sequences(valdf.toks,maxlen=max_len)
dataInputTest=sequence.pad_sequences(test.toks,maxlen=max_len)

In [101]:
' '.join([idx2word[i] for i in dataInputTrain[10,:]])

'<UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UN

In [112]:
sequence_input = Input(shape=(max_len, ))
x = Embedding(vocab_size+1, n_factors, input_length=max_len, weights=[emb],trainable = False)(sequence_input)
x = Bidirectional(LSTM(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = Concatenate()([avg_pool, max_pool])
preds = Dense(6, activation="sigmoid")(x)

In [113]:
model = Model(sequence_input, preds)

In [114]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 300, 300)     57097200    input_7[0][0]                    
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 300, 256)     439296      embedding_5[0][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 298, 64)      49216       bidirectional_3[0][0]            
__________________________________________________________________________________________________
global_ave

In [115]:
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3))

In [130]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')
mcp_save = ModelCheckpoint('weights/lstm_mdl', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
tensor_board = TensorBoard(log_dir='./logs/run2', write_graph = False,)
roc_callback = ROCCallBack(validation_data = [dataInputVal, valdf[output_names].values])

In [131]:
model.fit(x = dataInputTrain,
         y = traindf[output_names].values,
         batch_size = 64, epochs = 200,
         validation_data = [dataInputVal, valdf[output_names].values],
         callbacks=[earlyStopping, mcp_save, reduce_lr_loss, tensor_board, roc_callback])

Train on 143613 samples, validate on 15958 samples
Epoch 1/200
143613/143613 [==============================] - 1708s 12ms/step - loss: 0.0400 - val_loss: 0.0412
Epoch 2/200
143613/143613 [==============================] - 1763s 12ms/step - loss: 0.0367 - val_loss: 0.0405
Epoch 3/200
143613/143613 [==============================] - 1712s 12ms/step - loss: 0.0335 - val_loss: 0.0418
Epoch 4/200
143613/143613 [==============================] - 1706s 12ms/step - loss: 0.0301 - val_loss: 0.0426
Epoch 5/200
143613/143613 [==============================] - 1705s 12ms/step - loss: 0.0265 - val_loss: 0.0448
Epoch 6/200
143552/143613 [============================>.] - ETA: 0s - loss: 0.0228
Epoch 00006: reducing learning rate to 0.00010000000474974513.
roc-auc_val: 0.986543297785
143613/143613 [==============================] - 1710s 12ms/step - loss: 0.0228 - val_loss: 0.0475
Epoch 7/200
143613/143613 [==============================] - 1710s 12ms/step - loss: 0.0157 - val_loss: 0.0496


In [122]:
valdf[output_names].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ..., 
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0]])

#### Submission

In [132]:
pred = model.predict(dataInputTest, verbose = 1)

153164/153164 [==============================] - 914s 6ms/step


In [143]:
for counter,i in enumerate(output_names):
    test[i] = pred[:,counter]

In [144]:
test[['id'] + output_names].to_csv('data/answers/lstm1.csv', index = False)